In [1]:
#################  第四版(tool_list) ##################
from xmlrpc.server import SimpleXMLRPCServer
import base64
dasiyIp = ['192.168.0.103',8080]
wangIp = ['192.168.0.128',8888]
s = SimpleXMLRPCServer(("192.168.0.77",8887))
class tmd:
    # 傳送工單
    def workOrder(self,a):
        print(a)
        return ['20230313','20230314','20230315','20230316','20230317']
    # 傳送表單(未使用)
    def form(self,a):
        print(a)
        return ['CP-00-07']
    # 傳送表單結構 & 內容
    def structure(self,a):
        import  pandas as pd 
        import json
        import random
        print(a)
        info_map= {}
        ## 讀取Excel建立表單架構(frame)
        data = pd.read_excel(a['order']+'.xlsx',sheet_name=a['sheet'],index_col=False,dtype={'value':str,'groupwidth':str})
        ## 將value的部分,填充為空字串,方便等等要進行分割
        values = {'value':""}
        data = data.fillna(value=values)
        dataG = data.groupby(["item",'row'])
        struList=[]
        result={}
        for (item,row),j in dataG:
            styleInfo = j.loc[:,"item":]
            styleList = list()
            styleList = [row.dropna(axis=0) for _,row in styleInfo.iterrows()]
            styleDicList = [h.to_dict() for h in (styleList)]
            for k in styleDicList:
                # 指定type 需要切割value,包成list
                if (k['type'] == 'C') or (k['type'] == 'E') or (k['type'] == 'H')or (k['type'] == 'J'):
                    if ',' in k['value']:
                        k['value']=k['value'].split(",")
                # typeC 當有groupwidth時,需切成list
                if 'groupwidth' in k:
                    if ',' in k['groupwidth']:
                        k['groupwidth']=k['groupwidth'].split(",")
            if item in result:
                result[item].append(styleDicList)
            else:
                result[item] = [styleDicList]
        struList= list(result.values())
        # 隨機決定是否該次表單是否有內容(data), 1 update 0 new
        info_list = []
        random_int = 0
        # random_int = random.randint(0, 1)
        if random_int == 1:
            state = 'update'
            ## 讀取json獲取表單內容(data)       
            fn = 'jsondata.json'
            with open (fn) as fnobject:
                info_list = json.load(fnobject)[a['sheet']]['data']
            info_map = {'state':state,'frame':struList,'data':info_list}
        else:
            state = 'new'
            info_map = {'state':state,'frame':struList}
        return info_map
    # 接收表單內容      
    def formArchive(self,a):
        import json
        info_list={}
        fn = 'jsondata.json'
        with open (fn) as fnobject:
            info_list= json.load(fnobject)
        info_list[a['sheet']]=a
        with open (fn,'w') as fnobject:
            json.dump(info_list,fnobject,indent=2)
        print(a['sheet'])
        print(a)
        print('success')
        return 'Success'
    # 歷史紀錄
    def history(self,a):
        print(a)
        print('success')
        return 'Success'
    def testapi(self,a):
        print(a)
        with open('./123.xlsx', 'rb') as f:
            excel_content = f.read()
        return base64.b64encode(excel_content).decode('utf-8')
    def login(self,a):
        try:
            print(a)
            if(a['password']=='000'):
                return 'Success'
            return 'Failure'
        except:
            return 'Failure'
andd = tmd()
s.register_function(andd.login)
s.register_function(andd.testapi)
s.register_function(andd.workOrder)
s.register_function(andd.form)
s.register_function(andd.structure)
s.register_function(andd.formArchive)
s.register_function(andd.history)
s.serve_forever()

{'account': '346356', 'password': '000'}


192.168.0.77 - - [10/Jul/2023 14:15:51] "POST / HTTP/1.1" 200 -


{'account': '346356', 'password': '000dfdd'}


192.168.0.77 - - [10/Jul/2023 14:15:57] "POST / HTTP/1.1" 200 -
